# Ad-Hoc Commands

# Ansible Configuration

In [1]:
cd ~/src/Classes/Ansible

/Users/rick446/src/Classes/Ansible


In [2]:
cat ansible.cfg

[defaults]
inventory = ./.vagrant/provisioners/ansible/inventory/vagrant_ansible_inventory
# inventory = ./inventory
# inventory = ./digital_ocean.py
# remote_user = root
remote_user = rick
private_key_file = ./training-ssh-key
pipelining = False
vault_password_file = ./vault-password.txt

host_key_checking = False
local_tmp = ./.ansible/tmp
roles_path = ./roles
library = ./modules
fact_caching = jsonfile
fact_caching_connection = ./cachedir
gathering = smart



In [3]:
cat ./.vagrant/provisioners/ansible/inventory/vagrant_ansible_inventory

# Generated by Vagrant

db ansible_host=127.0.0.1 ansible_port=2200 ansible_user='vagrant' ansible_ssh_private_key_file='/Users/rick446/src/Classes/Ansible/.vagrant/machines/db/virtualbox/private_key'
web ansible_host=127.0.0.1 ansible_port=2222 ansible_user='vagrant' ansible_ssh_private_key_file='/Users/rick446/src/Classes/Ansible/.vagrant/machines/web/virtualbox/private_key'

[webdb]
web
db

[webdb:vars]
ntp_server=ntp.atlanta.example.com
proxy=proxy.atlanta.example.com


# Commands Modules

- command
- expect
- raw
- script
- shell
- telnet

We've already seen the `command` (default) module:

In [4]:
%%bash
ansible web -m command -a 'hostname'

web | SUCCESS | rc=0 >>
web


The `raw` module can be used to 'bootstrap' Ansible:

In [5]:
%%bash
# -- become is technically unnecessary here because we set the default user to be 'root'
ansible all --become -m raw -a "apt install -y python python-simplejson"

web | SUCCESS | rc=0 >>




python is already the newest version.
python-simplejson is already the newest version.
python-simplejson set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Shared connection to 127.0.0.1 closed.

db | SUCCESS | rc=0 >>




python is already the newest version.
python-simplejson is already the newest version.
python-simplejson set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Shared connection to 127.0.0.1 closed.



We can use the `shell` module to execute commands in a particular shell:

In [2]:
cd ..

/Users/rick446/src/Classes/Ansible


In [1]:
%%bash
ansible localhost -m shell -a 'echo $(hostname) $USER'

bash: line 1: ansible: command not found


In [4]:
%%bash
ansible localhost -m shell -a 'executable=/bin/sh echo $(hostname) $USER'

localhost | SUCCESS | rc=0 >>
faramir.local rick446


# Misc. Modules

## Ping

Check to see if the host is up and the Ansible system is working there

In [5]:
%%bash
ansible all -m ping 

web | SUCCESS => {
    "changed": false,
    "failed": false,
    "ping": "pong"
}
db | SUCCESS => {
    "changed": false,
    "failed": false,
    "ping": "pong"
}


# Setup

Gather facts about the host

In [6]:
%%bash
ansible localhost -m setup

localhost | SUCCESS => {
    "ansible_facts": {
        "ansible_all_ipv4_addresses": [
            "10.118.248.231"
        ],
        "ansible_all_ipv6_addresses": [
            "fe80::7450:7ff:fea1:e38e%awdl0",
            "fe80::e194:ad4f:39d:5d70%utun0"
        ],
        "ansible_apparmor": {
            "status": "disabled"
        },
        "ansible_architecture": "x86_64",
        "ansible_awdl0": {
            "device": "awdl0",
            "flags": [
                "UP",
                "BROADCAST",
                "RUNNING",
                "PROMISC",
                "SIMPLEX",
                "MULTICAST"
            ],
            "ipv4": [],
            "ipv6": [
                {
                    "address": "fe80::7450:7ff:fea1:e38e%awdl0",
                    "prefix": "64",
                    "scope": "0x9"
                }
            ],
            "macaddress": "76:50:07:a1:e3:8e",
            "media": "Unknown",
            "media_select": "autoselect",
    

# `copy`, `file`

Manipulate files on the host

In [7]:
%%bash
ansible web -b -m copy -a "src=ansible.cfg dest=/root/ansible.cfg"
ansible web -b -m file -a "dest=/root/ansible.cfg mode=0600 owner=nobody"

web | SUCCESS => {
    "changed": true,
    "checksum": "25ce25426d737c4834c243d5d645106abb4e4766",
    "dest": "/root/ansible.cfg",
    "failed": false,
    "gid": 0,
    "group": "root",
    "md5sum": "d8b675768504445c6334afa0c91f0f5a",
    "mode": "0644",
    "owner": "root",
    "size": 691,
    "src": "/home/vagrant/.ansible/tmp/ansible-tmp-1513104057.594638-226391247449081/source",
    "state": "file",
    "uid": 0
}
web | SUCCESS => {
    "changed": true,
    "failed": false,
    "gid": 0,
    "group": "root",
    "mode": "0600",
    "owner": "nobody",
    "path": "/root/ansible.cfg",
    "size": 691,
    "state": "file",
    "uid": 65534
}


## `template` - generate text files

We can use [Jinja2 templates](http://jinja.pocoo.org/docs/dev/templates/) with the `template` module. 

In [8]:
cat templates/welcome.j2

Hello there, {{myname}}.

{% for x in [1,2,3] %}
You are awesome!
{% endfor %}

That is all.

In [10]:
%%bash
ansible web -m template -a 'src=templates/welcome.j2 dest=~/welcome.txt' -e myname=Rick 

web | SUCCESS => {
    "changed": true,
    "checksum": "4148ad70de2ebec059fbde4203925b33300965f8",
    "dest": "/home/vagrant/welcome.txt",
    "failed": false,
    "gid": 1000,
    "group": "vagrant",
    "md5sum": "8ef44ff0bab99fe29478829b3da9bf2d",
    "mode": "0664",
    "owner": "vagrant",
    "size": 85,
    "src": "/home/vagrant/.ansible/tmp/ansible-tmp-1513104182.6464572-132264241412900/source",
    "state": "file",
    "uid": 1000
}


# Packaging Modules: Control installed packages

http://docs.ansible.com/ansible/latest/list_of_packaging_modules.html

In [11]:
%%bash
ansible web -b -m apt -a 'name=nginx state=latest'

web | SUCCESS => {
    "cache_update_time": 1512759465,
    "cache_updated": false,
    "changed": true,
    "failed": false,
    "stderr": "",
    "stderr_lines": [],
    "stdout": "Reading package lists...\nBuilding dependency tree...\nReading state information...\nThe following extra packages will be installed:\n  libxslt1.1 nginx-common nginx-core\nSuggested packages:\n  fcgiwrap nginx-doc\nThe following NEW packages will be installed:\n  libxslt1.1 nginx nginx-common nginx-core\n0 upgraded, 4 newly installed, 0 to remove and 0 not upgraded.\nNeed to get 495 kB of archives.\nAfter this operation, 1802 kB of additional disk space will be used.\nGet:1 http://archive.ubuntu.com/ubuntu/ trusty-updates/main libxslt1.1 amd64 1.1.28-2ubuntu0.1 [145 kB]\nGet:2 http://archive.ubuntu.com/ubuntu/ trusty-updates/main nginx-common all 1.4.6-1ubuntu3.8 [19.1 kB]\nGet:3 http://archive.ubuntu.com/ubuntu/ trusty-updates/main nginx-core amd64 1.4.6-1ubuntu3.8 [325 kB]\nGet:4 http://archive.ubuntu.co

`user` to create users
`authorized_key` to add ssh keys

In [24]:
%%bash
ansible web -b -m user -a 'name=rick'
ansible web -b -m authorized_key -a 'user=rick state=present key=https://raw.githubusercontent.com/Arborian/ansible-class/master/training-ssh-key.pub'

web | SUCCESS => {
    "changed": true,
    "comment": "",
    "createhome": true,
    "failed": false,
    "group": 1002,
    "home": "/home/rick",
    "name": "rick",
    "shell": "",
    "state": "present",
    "system": false,
    "uid": 1002
}
web | SUCCESS => {
    "changed": true,
    "comment": null,
    "exclusive": false,
    "failed": false,
    "key": "https://raw.githubusercontent.com/Arborian/ansible-class/master/training-ssh-key.pub",
    "key_options": null,
    "keyfile": "/home/rick/.ssh/authorized_keys",
    "manage_dir": true,
    "path": null,
    "state": "present",
    "unique": false,
    "user": "rick",
    "validate_certs": true
}


`user` for creating/managing users

In [17]:
%%bash
ansible web -b -m user -a 'name=rick state=absent remove=true'
ansible web -b -m file -a 'name=/home/rick state=absent'

web | SUCCESS => {
    "changed": false,
    "failed": false,
    "name": "rick",
    "state": "absent"
}
web | SUCCESS => {
    "changed": true,
    "failed": false,
    "path": "/home/rick",
    "state": "absent"
}


In [33]:
%%bash
ansible web -b -m user -a 'name=rick shell=/bin/bash'
ansible web -b -m authorized_key \
    -a 'user=rick state=present key=https://raw.githubusercontent.com/Arborian/ansible-class/master/training-ssh-key.pub'

web | SUCCESS => {
    "append": false,
    "changed": false,
    "comment": "",
    "group": 1002,
    "home": "/home/rick",
    "move_home": false,
    "name": "rick",
    "shell": "/bin/bash",
    "state": "present",
    "uid": 1002
}
web | SUCCESS => {
    "changed": true,
    "comment": null,
    "exclusive": false,
    "key": "https://raw.githubusercontent.com/Arborian/ansible-class/master/training-ssh-key.pub",
    "key_options": null,
    "keyfile": "/home/rick/.ssh/authorized_keys",
    "manage_dir": true,
    "path": null,
    "state": "present",
    "unique": false,
    "user": "rick",
    "validate_certs": true
}


`git` for deploying from source control

In [19]:
%%bash
ansible web -b -m apt -a 'name=git state=latest'
ansible web -m git -b --become-user=rick -a 'repo=https://github.com/Arborian/ansible-class.git dest=/home/rick/ansible-class'

web | SUCCESS => {
    "cache_update_time": 1512759465,
    "cache_updated": false,
    "changed": false,
    "failed": false
}
web | SUCCESS => {
    "after": "ecfddfa00dbba0a7d5e5a97172982e2fbfd54274",
    "before": null,
    "changed": true,
    "failed": false
}


# Aside

https://github.com/openstack/ara is a nice tool for viewing playbook runs

#  Managing Services

You can ensure that services are started and running using the `service` module:

In [2]:
%%bash
ansible web --become --become-user rick -m service -a 'name=nginx state=started'

bash: line 1: ansible: command not found


# Lab: Ad-hoc Commands

Using Ansible one-off commands:

- Create a user for yourself on the `web` host
- Make sure you can use the training key to ssh into the hosts as your user without a password
- Check out the class repository into your user's account
- Install and start nginx in the `web` host
- Verify that it is working by visiting http://localhost:8080